In [ ]:
import csv, sys, copy, random
import networkx as nx
import pandas as pd
import numpy as np
import scipy.stats as st

def list_d(G, seeds):
  if type(G) == nx.MultiGraph or type(G) == nx.MultiDiGraph:
      raise Exception("not defined for graphs with multiedges")

  for s in seeds:
    if int(s) not in G.nodes():
     raise Exception("seed", s, "is not in graph")

  # perform diffusion
  A = copy.deepcopy(seeds)
  return _diffuse_inf(G, A)

def _diffuse_inf(G, A):
  tried_edges = set()
  layer_i_nodes = [ ]
  layer_i_nodes.append([i for i in A])
  while True:
    len_old = len(A)
    (A, activated_nodes_of_this_round, cur_tried_edges) = \
        _diffuse_one_cascade(G, A, tried_edges)
    layer_i_nodes.append(activated_nodes_of_this_round)
    tried_edges = tried_edges.union(cur_tried_edges)
    if len(A) == len_old:
      break
  return layer_i_nodes

def _diffuse_one_cascade(G, A, tried_edges):
  activated_nodes_of_this_round = set()
  cur_tried_edges = set()
  for s in A:
    for nb in G.successors(s):
      if nb in A or (s, nb) in tried_edges or (s, nb) in cur_tried_edges:
        continue
      if _ppinf_success(G, s, nb):
        activated_nodes_of_this_round.add(nb)
      cur_tried_edges.add((s, nb))
  activated_nodes_of_this_round = list(activated_nodes_of_this_round)
  A.extend(activated_nodes_of_this_round)
  return A, activated_nodes_of_this_round, cur_tried_edges

def _ppinf_success(G, src, dest):
  return random.random() <= G[src][dest]['Rating']


#=========================================================

data = pd.read_csv('soc-bitcoinotc.csv')
G = nx.from_pandas_edgelist(data, 'Source', 'Target', ['Rating'], create_using=nx.DiGraph)
G_nodes = G.number_of_nodes()
G_edges = G.number_of_edges()

node_list = G.nodes()

# change to directed graph
if not G.is_directed():
    DG = G.to_directed()
else:
    DG = copy.deepcopy(G)

for e in DG.edges():
  DG[e[0]][e[1]]['Rating'] += 10

for n in DG.nodes():
  if 1==1:
    total_in_weight = 0
    for p in G.successors(n):
      total_in_weight += DG[n][p]['Rating']

    p=0
    for p in G.successors(n):
      if total_in_weight ==0:
        DG[n][p]['Rating'] = 0
      else:
        DG[n][p]['Rating'] = round((DG[n][p]['Rating']/total_in_weight)*100,1)

no_of_iterations = 100
seeds_DC = [35,2642,1810,2125,2028,905,4172,7,1,4197,13,1018,2296,1953,2388,4291,1334,546,1386,3988,2067,2045,1396,1352,1899,3735,2942,202,3897,3129,1565,1317,2625,3451,304,3649,3828,41,2266,1566,1832,4559,135,1731,353,832,2600,1383,1217,1585]

for step_no in range (1,6): # Five cascades
  spread_counts = [] # Initialize a list to store spread counts for each iteration
  for x in range(no_of_iterations):
      layer = list_d(DG, seeds_DC)
      tot_cnt = 0
      for f in layer:
          tot_cnt += len(f)

      spread_counts.append(tot_cnt) # Append the total spread count to the list

  # Calculate mean and standard deviation
  spread_array = np.array(spread_counts)
  mean_spread = np.mean(spread_array)
  std_spread = np.std(spread_array, ddof=1)  # sample standard deviation

  # Print results
  print(f"Step = {step_no}")
  print(f"  Average Spread       = {mean_spread:.2f}")
  print(f"  Standard Deviation   = {std_spread:.2f}")
  print('Step =',step_no, round((mean_spread),0)) # Use mean_spread for the rounded output